<a href="https://colab.research.google.com/github/hakonornh/Stundatafla/blob/main/Stundatafla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Stundatafla**

Markmið verkefnis er að búa til stundatöflu sem uppfyllir okkar kröfur.

Ákvörðunarbreytur (e. decision variables) eru:

$N$: fjöldi námskeiða

$S$: fjöldi stofa

$Sn$: stærð stofu

$H$: fjöldi hólfa

$D$: fjöldi vikudaga

$B$: byggingar

Skorður:

Hver áfangi á tvö slot í viku

$$\sum_{s\in S,h\in H,d\in D} y_{n,s,d,h} \leq 2 \quad \forall n\in N$$

Engir tímar fyrir kl. 10 á mánudögum og eftir hádegi á föstudögum

$$y_{n,s,1,1} = 0 \quad \forall n \in N, \forall s \in S$$

$$y_{n,s,5,3} = 0 \quad \forall n \in N, \forall s \in S$$

Það geta ekki verið tveir tímar á sama tíma

$$\sum_{n\in N} y_{n,s,d,h}\leq 1  \quad \forall n\in N$$

Ekki er hægt að tvíbóka stofu (ATH?)

$$\sum_{s\in S} y_{n,s,d,h}\leq 1 \quad \forall n\in N, d\in D, h\in H$$

Sami áfangi má ekki vera kenndur tvisvar sinnum sama dag

$$\sum_{n\in N,d\in D} y_{n,d}\leq 1 $$







In [ ]:
!python -m pip install -i https://pypi.gurobi.com gurobipy
import gurobipy as gp
from gurobipy import GRB
import numpy as np

Looking in indexes: https://pypi.gurobi.com


In [ ]:
# Byggingar (B) og Stofur (S) ásamt stærð stofu (Sn)
B = { "Askja" : ["N-132","N-121","N-129","N-130","N-120","N-128","N-131"], 
      "Logberg": ["L-101","L-102","L-103"], 
      "VR-II": ["V02-262","V02-147","V02-352","V02-354","V02-156","V02-138","V02-258","V02-261","V02-151","V02-148","V02-158","V02-157","V02-152","V02-155"],
      "Haskolabio" : ["HB-2","HB-3","HB-1"], 
      "Oddi" : ["O-101","O-201","O-202"], 
      "Taeknigardur" : ["Tg-227"], 
      "Naustid" : ["Naustid"], 
      "Haskolatorg": ["HT-101","HT-104","HT-103","HT-105","HT-102"],
      "Arnagardur" : ["Ag-201","Ag-301","Ag-304","Ag-311","Ag-422"]}
S = []
for b in B.keys():
  S.extend(B[b])
Sn = {
    "V02-147": 30,
    "V02-258": 34,
    "N-131": 55,
    "V02-262": 40,
    "V02-261": 60,
    "N-120": 15,
    "Tg-227": 20,
    "O-202": 40,
    "HB-2": 203,
    "V02-352": 28,
    "HT-104": 100,
    "O-101": 93,
    "N-129": 25,
    "V02-155": 38,
    "HT-105": 180,
    "N-130": 35,
    "Ag-422": 66,
    "O-201": 88,
    "V02-138": 22,
    "V02-354": 40,
    "HB-1": 296,
    "V02-157": 75,
    "V02-151": 12,
    "V02-156": 42,
    "HT-102": 180,
    "V02-158": 75,
    "L-103": 50,
    "HT-101": 60,
    "Ag-311": 52,
    "N-132": 150,
    "HT-103": 100,
    "N-128": 24,
    "N-121": 22,
    "HB-3": 142,
    "Ag-201": 86,
    "L-102": 50,
    "L-101": 106,
    "Ag-304": 40,
    "Ag-301": 86,
    "Naustid": 28
}
S = [s for s in Sn.keys()]


In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()

Saving fjoldinema.xlsx to fjoldinema (1).xlsx
Saving namsleidir.xlsx to namsleidir (1).xlsx


In [ ]:
# Skráningartölur (fjöldi nema) og tenging námskeiða (já/nei eða 1/0) í námsbrautum
# Gögnin er í Excel skjölum sem heita fjoldinema.xlsx og namsleidir.xlsx (sjá Canvas)
import pandas as pd

A = pd.read_excel('fjoldinema.xlsx', index_col='Unnamed: 0')
L = pd.read_excel('namsleidir.xlsx', index_col='Unnamed: 0')
# Námskeiðin eru:
N = [a for a in A.index if not "TÆK" in a]



In [ ]:
#gefnar
d = ['Mánudagur','Þriðjudagur','Miðvikudagur','Fimmtudagur','Föstudagur'] #dagar
h = [1,2,3]
S = [s for s in Sn.keys()]

m = gp.Model('Stundatafla')

#Breytur
y = m.addVars(S,d,h,N, vtype = 'B')

#Skorður
m.addConstrs(gp.quicksum(y[stofur,dagar,holf,namskeid] for stofur in S for dagar in d for holf in h) <= 2 for namskeid in N) #fyrir hvert námskeið eru tvö hólf í viku
m.addConstrs(gp.quicksum(y[stofur, dagar, holf, namskeid] for stofur in S for dagar in d for namskeid in N) <= 1 for holf in h) #Ekki tveir tímar í sama hólf
m.addConstrs(gp.quicksum(y[stofur, dagar, holf, namskeid] for dagar in D for holf in h for namskeid in N) <= 1 for stofur in S) #ekki hægt að tvíbóka stofur
m.addConstrs(gp.quicksum(y[dagar,namskeid] for dagar in d) <=1 for namskeid in N)

#Optimze
#m.optimize('Stundatafla')


KeyError: ignored

Þetta hérna að neðan er eitthvað sem við gætum mögulega notað til að sýna úrlausnina okkar! EKKI NAUÐSYNLEGT

In [ ]:
# Dæmi um hvernig væri hægt að vísa í gögnin:
print("Fjöldi námskeiða í boði hjá VON eru ", len(N))
# Skoðum árekstur og tengingu námsskeiða út frá námsbrautum
heiti = '5051IÐN508M20216' # okkar námskeið
for namskeid in N:
    if A.loc[heiti,namskeid] > 0 and L.loc[heiti,namskeid] == 1:
        print("Á námsbraut með ", heiti, " : ", namskeid, " fjöldi er ", A.loc[heiti,namskeid])
    elif A.loc[heiti,namskeid] > 0:
        print("Valnámskeið með ", heiti, " : ", namskeid, " fjöldi er ", A.loc[heiti,namskeid])

Fjöldi námskeiða í boði hjá VON eru  296
Valnámskeið með  5051IÐN508M20216  :  5055EFN301G20216  fjöldi er  1
Valnámskeið með  5051IÐN508M20216  :  5055EÐL102G20216  fjöldi er  1
Valnámskeið með  5051IÐN508M20216  :  5055STÆ104G20216  fjöldi er  1
Valnámskeið með  5051IÐN508M20216  :  5055STÆ302G20216  fjöldi er  2
Á námsbraut með  5051IÐN508M20216  :  5051IÐN508M20216  fjöldi er  32
Valnámskeið með  5051IÐN508M20216  :  5051IÐN113F20216  fjöldi er  1
Valnámskeið með  5051IÐN508M20216  :  5051IÐN303G20216  fjöldi er  6
Valnámskeið með  5051IÐN508M20216  :  5051IÐN102M20216  fjöldi er  2
Á námsbraut með  5051IÐN508M20216  :  5051IÐN502G20216  fjöldi er  27
Á námsbraut með  5051IÐN508M20216  :  5051IÐN509M20216  fjöldi er  30
Valnámskeið með  5051IÐN508M20216  :  5051IÐN103G20216  fjöldi er  2
Valnámskeið með  5051IÐN508M20216  :  5051IÐN302G20216  fjöldi er  2
Valnámskeið með  5051IÐN508M20216  :  5051TÖL107G20216  fjöldi er  1
Valnámskeið með  5051IÐN508M20216  :  5051TÖL101G20216  fjö

In [ ]:
# Skoða myndrænt hvernig námsleiðir tengjast
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

G = nx.from_pandas_adjacency(L > 0)
node_locs = nx.circular_layout(G)
theta = {k: np.arctan2(v[1], v[0]) * 180/np.pi for k, v in node_locs.items() }
plt.figure(figsize=(20,20))
nx.draw_networkx_nodes(G, pos=node_locs, alpha=.5, node_size=2)
labels = nx.draw_networkx_labels(G, pos=node_locs, font_size=8)
for key,t in labels.items():
    if 90 < theta[key] or theta[key] < -90 :
        angle = 180 + theta[key]
        t.set_ha('right')
    else:
        angle = theta[key]
        t.set_ha('left')
    t.set_va('center')
    t.set_rotation(angle)
    t.set_rotation_mode('anchor')
    t.set_text(t.get_text()[4:11])

nx.draw_networkx_edges(G, pos=node_locs, alpha=.4)
plt.box("off")
plt.xlim(-1.2,1.2)
plt.ylim(-1.2,1.2)

plt.show()

In [ ]:
# Dæmi um hvernig væri hægt að búa til stundatöflu með Gurobi lausn (y), sjá líka Wolsey 1.7 dæmi, hlekk á Canvas:
def soln_to_pandas(ns, y, N, H, D, S):
    df = pd.DataFrame(index=H, columns=D)
    for n in ns:
        for d in D:
            for h in H:
                for s in S:
                    if y[n,h,d,s].X > 0:
                      df.iloc[t,d] = n[4:11]+"("+s+")"
    df = df.rename_axis('Tímahólf').rename_axis('Vikudagur', axis='columns')
    return df
df = soln_to_pandas(['5051IÐN508M20216','5051IÐN502G20216','5051IÐN509M20216','5051IÐN510M20216','5051IÐN101M20216','5051IÐN303G20216'],y,N,H,D,S)
df